In [1]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/88.0.4324.96/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\angela\.wdm\drivers\chromedriver\win32\88.0.4324.96]


### Visit the NASA Mars News Site

In [32]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [33]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
news_soup = soup(html, 'html.parser')

slide_elem = news_soup.select_one('ul.item_list li.slide')

In [34]:
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8846/could-the-surface-of-phobos-reveal-secrets-of-the-martian-past/" target="_self">Could the Surface of Phobos Reveal Secrets of the Martian Past?</a></div>

In [35]:
# Use the parent element to find the first a tag and save it as `news_title`
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

'Could the Surface of Phobos Reveal Secrets of the Martian Past?'

In [36]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p

'The Martian moon Phobos orbits through a stream of charged atoms and molecules that flow off the Red Planet’s atmosphere, new research shows.'

### JPL Space Images Featured Image

In [37]:
try:
    PREFIX = "https://web.archive.org/web/20181114023740"
    url = f'{PREFIX}/https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
    browser.visit(url)
    article = browser.find_by_tag('article').first['style']
    article_background = article.split("_/")[1].replace('");',"")
    print(f'{PREFIX}_if/{article_background}')
except:
    print('https://www.nasa.gov/sites/default/files/styles/full_width_feature/public/thumbnails/image/pia22486-main.jpg')

https://web.archive.org/web/20181114023740_if/https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA13664-1920x1200.jpg


In [38]:
# Find and click the full image button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [39]:
# Find the more info button and click that
browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.links.find_by_partial_text('more info')
more_info_elem.click()

In [40]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [41]:
# find the relative image url
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel

AttributeError: 'NoneType' object has no attribute 'get'

In [42]:
# Use the base url to create an absolute url
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

'https://www.jpl.nasa.govhttps://www.jpl.nasa.gov/spaceimages/images/largesize/PIA13664_hires.jpg'

### Mars Facts

In [43]:
df = pd.read_html('http://space-facts.com/mars/')[0]

df.head()

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [44]:
df.columns=['Description', 'Mars']
df.set_index('Description', inplace=True)
df

,Mars
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [45]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n

### Mars Weather

In [46]:
# Visit the weather website
url = 'https://mars.nasa.gov/insight/weather/'
browser.visit(url)

In [47]:
# Parse the data
html = browser.html
weather_soup = soup(html, 'html.parser')

In [48]:
# Scrape the Daily Weather Report table
weather_table = weather_soup.find('table', class_='mb_table')
print(weather_table.prettify())

<table class="mb_table" id="weather_observation" style="width:100%;">
 <thead>
  <tr>
   <th colspan="2" scope="col">
    Time
   </th>
   <th colspan="3" id="temperature_lbl" scope="col">
    Air Temperature (
    <span class="lbl_fahrenheit">
     °F
    </span>
    <span class="slash">
     |
    </span>
    <span class="lbl_celsius fadeBlack">
     °C
    </span>
    )
   </th>
   <th colspan="4" id="windspeed_lbl" scope="col">
    Wind Speed (
    <span class="lbl_mph">
     mph
    </span>
    <span class="slash">
     |
    </span>
    <span class="lbl_mps fadeBlack">
     m/s
    </span>
    )
   </th>
   <th colspan="3" id="pressure_lbl" scope="col">
    Pressure (Pa)
   </th>
  </tr>
 </thead>
 <tbody>
  <tr id="weather_top">
   <th class="sol" scope="row">
    Date
   </th>
   <th class="sol" scope="row">
    Sol
   </th>
   <td class="temperature max">
    Max.
   </td>
   <td class="temperature avg">
    Avg.
   </td>
   <td class="temperature min">
    Min.
   </td>
   <t

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [3]:
# 1. Use browser to visit the URL 
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [4]:
browser.visit(url)

In [5]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

# 3. Write code to retrieve the image urls and titles for each hemisphere.
# parse site
#Cerberus
browser.visit(url)
browser.links.find_by_partial_text('Cerberus').click()
html = browser.html
cerberus_soup = soup(html, 'html.parser')
cerberus_url = cerberus_soup.select_one('div.downloads a').get("href")
cerberus_title = cerberus_soup.select_one('div.content h2').text

#dictionary:
cerberus_dict = {
        "img_url": cerberus_url,
        "title": cerberus_title
    }

hemisphere_image_urls.append(cerberus_dict)

In [7]:
#Schiaparelli
browser.visit(url)
browser.links.find_by_partial_text('Schiaparelli').click()
html = browser.html
schiaparelli_soup = soup(html, 'html.parser')
schiaparelli_url = schiaparelli_soup.select_one('div.downloads a').get("href")
schiaparelli_title = schiaparelli_soup.select_one('div.content h2').text

#dictionary:
schiaparelli_dict = {
        "img_url": schiaparelli_url,
        "title": schiaparelli_title
    }

hemisphere_image_urls.append(schiaparelli_dict)

In [8]:
#Syrtis
browser.visit(url)
browser.links.find_by_partial_text('Syrtis').click()
html = browser.html
syrtis_soup = soup(html, 'html.parser')
syrtis_url = syrtis_soup.select_one('div.downloads a').get("href")
syrtis_title = syrtis_soup.select_one('div.content h2').text

#dictionary:
syrtis_dict = {
        "img_url": syrtis_url,
        "title": syrtis_title
    }

hemisphere_image_urls.append(syrtis_dict)

In [9]:
#Valles
browser.visit(url)
browser.links.find_by_partial_text('Valles').click()
html = browser.html
valles_soup = soup(html, 'html.parser')
valles_url = valles_soup.select_one('div.downloads a').get("href")
valles_title = valles_soup.select_one('div.content h2').text

#dictionary:
valles_dict = {
        "img_url": valles_url,
        "title": valles_title
    }

hemisphere_image_urls.append(valles_dict)

In [10]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [11]:
# 5. Quit the browser
browser.quit()